## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [5]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSSSBSBS' # added BSSSBSBS for A08001117
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
65
+-----------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|          coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|          res_mean|    station|total_var|undefs|              vector|  year|
+-----------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|-0.86546243957038|0.4403268419370094|-0.3773956155870098|     63.4|BSSSBSBS| 30.6794| -88.2397|       SNWD|0.9987760367434622|0.9984592096944312|0.9982264725871203|0.9791473465320034|USW00013894|    6

In [6]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [7]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USW00013894,1958.0,0.440327
1,USW00013858,1958.0,0.440327
2,USC00013255,1958.0,0.440327
3,USC00013496,1958.0,0.440327
4,USC00015483,1958.0,0.440327


In [8]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00010252,USC00010583,USC00011080,USC00011084,USC00011566,USC00011685,USC00011803,USC00011807,USC00012577,USC00012675,...,USC00228382,USW00003855,USW00013838,USW00013846,USW00013858,USW00013894,USW00013899,USW00093828,USW00093841,USW00093859
year,,,,,,,,,,,,,,,,,,,,,
1954.0,NaN,NaN,NaN,0.435310,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.433288,NaN,NaN,NaN,NaN
1955.0,NaN,0.441121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-837.568350,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.446034,NaN,NaN
1958.0,NaN,NaN,NaN,NaN,NaN,NaN,0.443524,0.449669,NaN,NaN,...,NaN,0.443524,NaN,0.440327,0.440327,0.440327,NaN,NaN,0.434541,0.440327
1963.0,NaN,NaN,NaN,NaN,0.458337,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1964.0,NaN,NaN,NaN,NaN,0.535403,0.473131,NaN,NaN,NaN,NaN,...,NaN,NaN,0.452238,NaN,NaN,0.452238,NaN,NaN,NaN,NaN
1968.0,NaN,NaN,NaN,0.436177,0.437063,NaN,0.434147,NaN,NaN,0.433713,...,0.442783,NaN,NaN,-0.155452,NaN,NaN,NaN,NaN,NaN,NaN
1973.0,NaN,NaN,NaN,NaN,NaN,NaN,0.441131,0.451099,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.451503,0.444601,NaN,0.437912,NaN
1977.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.434542,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.432911,NaN,NaN,0.432911,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [9]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  103.888497924
RMS removing mean-by-station=  73.4992871268
RMS removing mean-by-year   =  84.8682991268


In [10]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 103.888497924
0 after removing mean by year    = 84.8682991268
0 after removing mean by stations= 63.2567301675
1 after removing mean by year    = 60.5112899235
1 after removing mean by stations= 59.3458969531
2 after removing mean by year    = 58.7327417369
2 after removing mean by stations= 58.3669096404
3 after removing mean by year    = 58.1291932369
3 after removing mean by stations= 57.9643639981
4 after removing mean by year    = 57.844133185
4 after removing mean by stations= 57.7529741728
